Ref:https://www.kaggle.com/errolpereira/xgboost-with-feature-engineering/data#Model-Building.

1. I followed above kernel. 

2. Earlier I was merging all dataframes and creating features. This time I used the same approach as reference. 

3.The features used in reference are the most common features so I too have them. Along with this I tried some new 5-6 features too.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Import Statements
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from scipy import stats
from tqdm import tqdm
import pandas as pd
import numpy as np
import pickle
import gc
import os

In [ ]:
# Importing each csv files
aisles = pd.read_csv('/content/drive/My Drive/Instacart Analysis/aisles.csv')

department = pd.read_csv('/content/drive/My Drive/Instacart Analysis/departments.csv')

orders = pd.read_csv('/content/drive/My Drive/Instacart Analysis/orders.csv')

products = pd.read_csv('/content/drive/My Drive/Instacart Analysis/products.csv')

prior = pd.read_csv('/content/drive/My Drive/Instacart Analysis/order_products_prior.csv')

train = pd.read_csv('/content/drive/My Drive/Instacart Analysis/order_products_train.csv')

submission = pd.read_csv('/content/drive/My Drive/Instacart Analysis/sample_submission.csv')

In [ ]:
# Will see the shape of each dataframe and will decide whether to drop some rows or not
print("Aisles DataFrame",aisles.shape)
print("Department DataFrame",department.shape)
print("Orders DataFrame",orders.shape)
print("Products DataFrame",products.shape)
print("Order Products Prior DataFrame",prior.shape)
print("Order Products Train DataFrame",train.shape)
print("Submission DataFrame",submission.shape)

Aisles DataFrame (134, 2)
Department DataFrame (21, 2)
Orders DataFrame (3421083, 7)
Products DataFrame (49688, 4)
Order Products Prior DataFrame (32434489, 4)
Order Products Train DataFrame (1384617, 4)
Submission DataFrame (75000, 2)


In [ ]:
orders.head(2)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0


In [ ]:
prior.head(2)

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1


In [ ]:
train.head(2)

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1


In [ ]:
# Since there are many NaN values in 'days_since_prior_order' so we do mean imputation for them
# Below code finds the mean of the 'days_since_prior_order

prior_order = list()
for key,value in enumerate(orders['days_since_prior_order'].values):
    if(str(value)=='nan'):
      flag=1
    else:  
      prior_order.append(value)
mean_value = sum(prior_order)/len(prior_order)
print("Mean Value Of Days Since Prior Order",np.round(mean_value,1))

# Replaces NaN with mean of 'days_since_prior_order'

prior_order = list()
null_count = 0
mean_values = np.round(mean_value,1)
for key,value in enumerate(orders['days_since_prior_order'].values):
    if(str(value)=='nan'):
        prior_order.append(mean_values)
        null_count+=1
    else:
        prior_order.append(value)           
print("Total Null Values were", null_count)        

orders.drop('days_since_prior_order', axis=1, inplace=True)
orders['days_since_prior_order'] = prior_order

Mean Value Of Days Since Prior Order 11.1
Total Null Values were 206209


In [ ]:
prior_order_df = pd.merge(orders, prior, on='order_id', how='inner')
prior_order_df.head(3)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,11.1,196,1,0
1,2539329,1,prior,1,2,8,11.1,14084,2,0
2,2539329,1,prior,1,2,8,11.1,12427,3,0


<h2> Creating User Features </h2>

1. Maximum number of order by user.
2. Mean number of order by user.
3. Minimum number of order by user.

In [ ]:
user_df = pd.DataFrame()
user_df['max_number_of_orders'] = prior_order_df.groupby(by='user_id')['order_number'].aggregate('max').astype(np.uint8)
user_df['mean_number_of_orders'] = prior_order_df.groupby(by='user_id')['order_number'].aggregate('mean').astype(np.uint8)
user_df['min_number_of_orders'] = prior_order_df.groupby(by='user_id')['order_number'].aggregate('min').astype(np.uint8)
print("Null values", user_df.isnull().any().sum().sum())
user_df.head(3)

Null values 0


,max_number_of_orders,mean_number_of_orders,min_number_of_orders
user_id,,,
1,10,5,1
2,14,8,1
3,12,6,1


<h2> Creating Max Order Features </h2>

1. Average product by user in each order

In [ ]:
max_order = pd.DataFrame()
average_products= pd.DataFrame()
max_order['total_products_per_order'] = prior_order_df.groupby(by=['user_id', 'order_id'])['product_id'].aggregate('count').astype(np.uint8)
average_products['average_user_product'] = max_order.groupby(by=['user_id'])['total_products_per_order'].aggregate('mean').astype(np.float16)

print("Null values", average_products.isnull().any().sum().sum())
average_products.head(3)

Null values 0


,average_user_product
user_id,
1,5.898438
2,13.929688
3,7.332031


<h2> Day Of Week Features</h2>

1. Maximum order by user day wise

In [ ]:
day_of_week = pd.DataFrame()
day_of_week['max_order_day'] = prior_order_df.groupby(by=['user_id'])['order_dow'].aggregate(lambda x : stats.mode(x)[0]).astype(np.uint8)
day_of_week = day_of_week.reset_index()
print("Null values", day_of_week.isnull().any().sum().sum())
day_of_week.head()

Null values 0


,user_id,max_order_day
0,1,4
1,2,2
2,3,0
3,4,4
4,5,3


<h2>Hour Of Day Feature</h2>

1. Hour with maximum order by user 

In [ ]:
hour_of_day = pd.DataFrame()
hour_of_day['max_order_by_hour'] = prior_order_df.groupby(by=['user_id'])['order_hour_of_day'].aggregate(lambda x : stats.mode(x)[0]).astype(np.uint8)
hour_of_day = hour_of_day.reset_index()
print("Total NUll Values",hour_of_day.isnull().any().sum().sum())
hour_of_day.head(3)

Total NUll Values 0


,user_id,max_order_by_hour
0,1,7
1,2,9
2,3,16


<h2>Reorder Features</h2>

1. Total Number Of Reorders.
2. Total Number Of Reorders Not Done.
3. Reorder Ratio For Each User

In [ ]:
reorder_by_user = pd.DataFrame()
reorder_by_user['total_reorders'] = prior_order_df[prior_order_df['reordered']==1].groupby('user_id')['reordered'].aggregate('count').astype(np.uint8)
reorder_by_user['total_non_reorders'] = prior_order_df[prior_order_df['reordered']==0].groupby('user_id')['reordered'].aggregate('count').astype(np.uint8)
reorder_by_user['reorder_ratio'] = reorder_by_user['total_reorders']/(reorder_by_user['total_reorders'] + reorder_by_user['total_non_reorders']).astype(np.float16)
print("Total NUll Values",reorder_by_user.isnull().any().sum().sum())
reorder_by_user.head(3)

Total NUll Values 0


,total_reorders,total_non_reorders,reorder_ratio
user_id,,,
1,41,18,0.694824
2,93,102,0.476807
3,55,33,0.625000


<h2>Merging All Features Created Till Now On 'User_id'</h2>

In [ ]:
user_df = pd.merge(user_df, average_products, on='user_id', how='left')  
user_df = pd.merge(user_df, day_of_week, on='user_id', how='left')
user_df = pd.merge(user_df, hour_of_day, on='user_id', how='left')
user_df = pd.merge(user_df, reorder_by_user, on='user_id', how='left')
print("Total null values", user_df.isnull().any().sum())
user_df.head(3)

Total null values 3


,user_id,max_number_of_orders,mean_number_of_orders,min_number_of_orders,average_user_product,max_order_day,max_order_by_hour,total_reorders,total_non_reorders,reorder_ratio
0,1,10,5,1,5.898438,4,7,41.0,18.0,0.694824
1,2,14,8,1,13.929688,2,9,93.0,102.0,0.476807
2,3,12,6,1,7.332031,0,16,55.0,33.0,0.625000


In [ ]:
#pickle.dump((user_df),open('/content/drive/My Drive/Instacart Analysis/user_df.pkl','wb'))
user_df = pickle.load(open('/content/drive/My Drive/Instacart Analysis/user_df.pkl','rb'))

<h2> Deleting DataFrame which are not more needed</h2>

In [ ]:
del average_products, day_of_week, hour_of_day, reorder_by_user, prior
user_df = user_df.fillna(0)
print("Total null values", user_df.isnull().any().sum())

Total null values 0


<h2>Product DataFrame</h2>

1. Number Of Times Product Reordered

In [ ]:
product_df = pd.DataFrame()
product_df['times_product_ordered'] = prior_order_df.groupby(by='product_id')['order_id'].aggregate('count').astype(np.uint16)
print("Total null values", product_df.isnull().any().sum().sum())
product_df.head(3)

Total null values 0


,times_product_ordered
product_id,
1,1852
2,90
3,277


<h2>Product Reorder Features</h2>

1. Product Reorder Ratio
2. Time Product Not Reordered

In [ ]:
reorder_products = pd.DataFrame()
reorder_products['product_reorder_ratio'] = prior_order_df.groupby('product_id')['reordered'].aggregate('mean').astype(np.float16)
reorder_products['product_not_reorder_time'] = prior_order_df[prior_order_df['reordered']==0].groupby('product_id')['reordered'].aggregate('count').astype(np.uint8)
print("Total null values",reorder_products.isnull().any().sum().sum())

reorder_products.head(3)

Total null values 0


,product_reorder_ratio,product_not_reorder_time
product_id,,
1,0.613281,204
2,0.133301,78
3,0.732910,74


<h2>Add To Cart Features</h2>

1. Average Position Of Product On Cart.
2. Late Position Of Product On Cart.
3. Early Position Of Product On Cart.

In [ ]:
add_to_cart = pd.DataFrame()
add_to_cart['average_product_cart_position'] = prior_order_df.groupby(by='product_id')['add_to_cart_order'].aggregate('mean').astype(np.float16)
add_to_cart['late_product_cart_position'] = prior_order_df.groupby(by='product_id')['add_to_cart_order'].aggregate('max').astype(np.float16)
add_to_cart['early_product_cart_position'] = prior_order_df.groupby(by='product_id')['add_to_cart_order'].aggregate('min').astype(np.float16)
print("Total null values",add_to_cart.isnull().any().sum().sum())
add_to_cart.head(3)

Total null values 0


,average_product_cart_position,late_product_cart_position,early_product_cart_position
product_id,,,
1,5.800781,74.0,1.0
2,9.890625,45.0,1.0
3,6.414062,41.0,1.0


<h2>Day Of Week Features</h2>

1. Average Nummber Of Product Order Day Wise
2. Earliest Week Day Product Ordered.
3. Late Week Day Product Ordered.

In [ ]:
dow_product = pd.DataFrame()
dow_product['average_product_order_day'] = prior_order_df.groupby(by='product_id')['order_dow'].aggregate('mean').astype(np.float16)
dow_product['late_product_order_day'] = prior_order_df.groupby(by='product_id')['order_dow'].aggregate('max').astype(np.float16)
dow_product['early_product_order_day'] = prior_order_df.groupby(by='product_id')['order_dow'].aggregate('min').astype(np.float16)
print("Total null values",dow_product.isnull().any().sum().sum())
dow_product.head(3)

Total null values 0


,average_product_order_day,late_product_order_day,early_product_order_day
product_id,,,
1,2.777344,6.0,0.0
2,2.921875,6.0,0.0
3,2.736328,6.0,0.0


<h2>Order Hour Of Day Features</h2>

1. Average Time Product Ordered.
2. Late Time Of Product Ordered.
3. Earliest Time Of Product Ordered.

In [ ]:
order_hour_of_day_product = pd.DataFrame()
order_hour_of_day_product['average_product_hour'] = prior_order_df.groupby(by='product_id')['order_hour_of_day'].aggregate('mean').astype(np.float16)
order_hour_of_day_product['late_product_hour'] = prior_order_df.groupby(by='product_id')['order_hour_of_day'].aggregate('max').astype(np.float16)
order_hour_of_day_product['early_product_hour'] = prior_order_df.groupby(by='product_id')['order_hour_of_day'].aggregate('min').astype(np.float16)
print("Total null values",order_hour_of_day_product.isnull().any().sum().sum())
order_hour_of_day_product.head(3)

Total null values 0


,average_product_hour,late_product_hour,early_product_hour
product_id,,,
1,13.234375,23.0,0.0
2,13.281250,22.0,0.0
3,12.101562,23.0,0.0


<h1>Merging all the product features on 'product_id'</h1>

In [ ]:
product_df = pd.merge(product_df, reorder_products, on='product_id', how='left')
product_df = pd.merge(product_df, add_to_cart, on='product_id', how='left')
product_df = pd.merge(product_df, dow_product, on='product_id', how='left')
product_df = pd.merge(product_df, order_hour_of_day_product, on='product_id', how='left')
print("Total null values",product_df.isnull().any().sum().sum())
product_df.head(3)

Total null values 0


,times_product_ordered,product_reorder_ratio,product_not_reorder_time,average_product_cart_position,late_product_cart_position,early_product_cart_position,average_product_order_day,late_product_order_day,early_product_order_day,average_product_hour,late_product_hour,early_product_hour
product_id,,,,,,,,,,,,
1,1852,0.613281,204,5.800781,74.0,1.0,2.777344,6.0,0.0,13.234375,23.0,0.0
2,90,0.133301,78,9.890625,45.0,1.0,2.921875,6.0,0.0,13.281250,22.0,0.0
3,277,0.732910,74,6.414062,41.0,1.0,2.736328,6.0,0.0,12.101562,23.0,0.0


In [ ]:
#pickle.dump((product_df),open('/content/drive/My Drive/Instacart Analysis/product_df.pkl','wb'))
product_df = pickle.load(open('/content/drive/My Drive/Instacart Analysis/product_df.pkl','rb'))

<h2>User Product Features</h2>

1. Times Product Ordered By User

In [ ]:
user_product_df = pd.DataFrame()
user_product_df['times_product_by_user'] = prior_order_df.groupby(by=['user_id', 'product_id'])['order_id'].aggregate('count').astype(np.uint8)
user_product_df = user_product_df.reset_index()
print("Total null values",user_product_df.isnull().any().sum().sum())
user_product_df.head(3)

Total null values 0


,user_id,product_id,times_product_by_user
0,1,196,10
1,1,10258,9
2,1,10326,1


<h2>Product Bought Features</h2>

1. Times Product Bought By User
2. Minimum Times Product Ordered By User

In [ ]:
product_bought = pd.DataFrame()
product_bought['times_bought'] = prior_order_df.groupby(by=['user_id', 'product_id'])['order_id'].aggregate('count').astype(np.uint8)
product_bought['earliest_order_position'] = prior_order_df.groupby(by=['user_id', 'product_id'])['order_number'].aggregate('min').astype(np.uint8)
product_bought = product_bought.reset_index()
print("Total null values",product_bought.isnull().any().sum().sum())
product_bought.head(3)

Total null values 0


,user_id,product_id,times_bought,earliest_order_position
0,1,196,10,1
1,1,10258,9,2
2,1,10326,1,5


<h2>Order By User Features</h2>

1. Total Order By User.

In [ ]:
order_by_user = pd.DataFrame()
order_by_user['total_orders_by_user'] = prior_order_df.groupby('user_id')['order_number'].aggregate('max').astype(np.uint16)
order_by_user = order_by_user.reset_index()
print("Total null values",order_by_user.isnull().any().sum().sum())
order_by_user.head(3)

Total null values 0


,user_id,total_orders_by_user
0,1,10
1,2,14
2,3,12


<h2>Merging Bought Product & Order By User</h2>

In [ ]:
merged_user_order = pd.merge(order_by_user, product_bought, how='right', on='user_id')
print("Total null values",merged_user_order.isnull().any().sum().sum())
merged_user_order.head(3)

Total null values 0


,user_id,total_orders_by_user,product_id,times_bought,earliest_order_position
0,1,10,196,10,1
1,1,10,10258,9,2
2,1,10,10326,1,5


<h2>Order Range Feature</h2>


In [ ]:
merged_user_order['order_range'] = merged_user_order['total_orders_by_user'] - merged_user_order['earliest_order_position'] + 1
print("Total null values",merged_user_order.isnull().any().sum().sum())
merged_user_order.head(3)

Total null values 0


,user_id,total_orders_by_user,product_id,times_bought,earliest_order_position,order_range
0,1,10,196,10,1,10
1,1,10,10258,9,2,9
2,1,10,10326,1,5,6


<h2>Merging User Product Features On 'user_id' and 'product_id'</h2>

1. Reorder ratio of product by user

In [ ]:
user_product_merged_df = pd.merge(merged_user_order, user_product_df, on=['user_id', 'product_id'], how='left')
user_product_merged_df['user_pro_reorder_ratio'] = user_product_merged_df['times_bought'] / user_product_merged_df['order_range']
print("Total null values",user_product_merged_df.isnull().any().sum().sum())
user_product_merged_df.head(3)

Total null values 0


,user_id,total_orders_by_user,product_id,times_bought,earliest_order_position,order_range,times_product_by_user,user_pro_reorder_ratio
0,1,10,196,10,1,10,10,1.000000
1,1,10,10258,9,2,9,9,1.000000
2,1,10,10326,1,5,6,1,0.166667


<h2>Dropping Features Which Aren't Required</h2>

In [ ]:
user_product_merged_df = user_product_merged_df.drop(['total_orders_by_user','order_range','times_product_by_user','earliest_order_position','times_bought'],axis=1)
user_product_merged_df.head(3)

,user_id,product_id,user_pro_reorder_ratio
0,1,196,1.000000
1,1,10258,1.000000
2,1,10326,0.166667


<h2>Dropping Features From Product Bought</h2>

In [ ]:
product_bought.drop(['earliest_order_position'],axis=1, inplace=True)
product_bought.head(3)

,user_id,product_id,times_bought
0,1,196,10
1,1,10258,9
2,1,10326,1


<h2>Merging Bought Product With User Product Features</h2>

In [ ]:
product_bought = pd.merge(product_bought, user_product_merged_df, on=['user_id','product_id'], how='left')
print("Total null values",product_bought.isnull().any().sum().sum())
product_bought.head(3)

Total null values 0


,user_id,product_id,times_bought,user_pro_reorder_ratio
0,1,196,10,1.000000
1,1,10258,9,1.000000
2,1,10326,1,0.166667


<h2>Order Number Back Features</h2>

1. Order Number Back: Times Product Bought By User In Recent Orders

In [ ]:
prior_order_df['order_number_back'] = (prior_order_df.groupby(by=['user_id'])['order_number'].transform(max) - prior_order_df['order_number'] + 1).astype(np.uint8)
print("Total null values", prior_order_df.isnull().any().sum().sum())
prior_order_df.head(3)

Total null values 0


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,order_number_back
0,2539329,1,prior,1,2,8,11.1,196,1,0,10
1,2539329,1,prior,1,2,8,11.1,14084,2,0,10
2,2539329,1,prior,1,2,8,11.1,12427,3,0,10


<h2>Times Product Bought By Use In Last 10 Orders</h2>

In [ ]:
temp = prior_order_df.loc[prior_order_df['order_number_back'] <= 10]
temp.head(3)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,order_number_back
0,2539329,1,prior,1,2,8,11.1,196,1,0,10
1,2539329,1,prior,1,2,8,11.1,14084,2,0,10
2,2539329,1,prior,1,2,8,11.1,12427,3,0,10


<h2>Recent Ten Orders</h2>

In [ ]:
recent_orders = pd.DataFrame()
recent_orders['recent_ten'] = (temp.groupby(by=['user_id', 'product_id'])['order_id'].aggregate('count')).astype(np.uint16)
recent_orders = recent_orders.reset_index()
recent_orders.head(3)

,user_id,product_id,recent_ten
0,1,196,10
1,1,10258,9
2,1,10326,1


<h2>Recent Ten Order Ratio</h2>

In [ ]:
recent_orders['recent_ten_ratio'] = (recent_orders['recent_ten'] / 10.0).astype(np.float16)
recent_orders.head(3)

,user_id,product_id,recent_ten,recent_ten_ratio
0,1,196,10,1.000000
1,1,10258,9,0.899902
2,1,10326,1,0.099976


<h2>Merging Product Bought & Recent Orders</h2>

In [ ]:
product_bought = pd.merge(product_bought, recent_orders, on=['user_id', 'product_id'], how='left')
del temp, recent_orders
print("Total null",product_bought.isnull().any().sum().sum())
product_bought.head(3)

Total null 2


,user_id,product_id,times_bought,user_pro_reorder_ratio,recent_ten,recent_ten_ratio
0,1,196,10,1.000000,10.0,1.000000
1,1,10258,9,1.000000,9.0,0.899902
2,1,10326,1,0.166667,1.0,0.099976


<h2>Since there are null values so filling them with 0</h2>

In [ ]:
#product_bought = product_bought.fillna(0)
#pickle.dump((product_bought),open('/content/drive/My Drive/Instacart Analysis/product_bought.pkl','wb'))
product_bought = pickle.load(open('/content/drive/My Drive/Instacart Analysis/product_bought.pkl','rb'))
product_bought.head(3)

,user_id,product_id,times_bought,user_pro_reorder_ratio,recent_ten,recent_ten_ratio
0,1,196,10,1.000000,10.0,1.000000
1,1,10258,9,1.000000,9.0,0.899902
2,1,10326,1,0.166667,1.0,0.099976


<h2>Merging DataFrame Which Contains All Features</h2>

In [ ]:
final_data = pd.merge(product_bought, user_df, on='user_id', how='left')
final_data = pd.merge(final_data, product_df, on='product_id', how='left')
print("Total null values",final_data.isnull().any().sum().sum())
final_data.head(3)

Total null values 0


,user_id,product_id,times_bought,user_pro_reorder_ratio,recent_ten,recent_ten_ratio,max_number_of_orders,mean_number_of_orders,min_number_of_orders,average_user_product,max_order_day,max_order_by_hour,total_reorders,total_non_reorders,reorder_ratio,times_product_ordered,product_reorder_ratio,product_not_reorder_time,average_product_cart_position,late_product_cart_position,early_product_cart_position,average_product_order_day,late_product_order_day,early_product_order_day,average_product_hour,late_product_hour,early_product_hour
0,1,196,10,1.000000,10.0,1.000000,10,5,1,5.898438,4,7,41.0,18.0,0.694824,35791,0.776367,64,3.722656,60.0,1.0,2.898438,6.0,0.0,12.523438,23.0,0.0
1,1,10258,9,1.000000,9.0,0.899902,10,5,1,5.898438,4,7,41.0,18.0,0.694824,1946,0.713867,45,4.277344,29.0,1.0,2.718750,6.0,0.0,12.257812,23.0,0.0
2,1,10326,1,0.166667,1.0,0.099976,10,5,1,5.898438,4,7,41.0,18.0,0.694824,5526,0.651855,131,4.191406,37.0,1.0,2.742188,6.0,0.0,12.898438,23.0,0.0


In [ ]:
del product_bought, product_df, user_df

In [ ]:
#pickle.dump((final_data),open('/content/drive/My Drive/Instacart Analysis/final_data.pkl','wb'))
final_data = pickle.load(open('/content/drive/My Drive/Instacart Analysis/final_data.pkl','rb'))

<h1>Creating Train Test Data</h1>

In [ ]:
train_test_data = orders.loc[((orders['eval_set'] == 'train') | (orders['eval_set'] == 'test')), ['user_id', 'eval_set', 'order_id']]
train_test_data.head()

,user_id,eval_set,order_id
10,1,train,1187899
25,2,train,1492625
38,3,test,2774568
44,4,test,329954
49,5,train,2196797


In [ ]:
final_data = pd.merge(final_data, train_test_data, on='user_id', how='left')
print("Total null values",final_data.isnull().any().sum().sum())
final_data.head(3)

Total null values 0


,user_id,product_id,times_bought,user_pro_reorder_ratio,recent_ten,recent_ten_ratio,max_number_of_orders,mean_number_of_orders,min_number_of_orders,average_user_product,max_order_day,max_order_by_hour,total_reorders,total_non_reorders,reorder_ratio,times_product_ordered,product_reorder_ratio,product_not_reorder_time,average_product_cart_position,late_product_cart_position,early_product_cart_position,average_product_order_day,late_product_order_day,early_product_order_day,average_product_hour,late_product_hour,early_product_hour,eval_set,order_id
0,1,196,10,1.000000,10.0,1.000000,10,5,1,5.898438,4,7,41.0,18.0,0.694824,35791,0.776367,64,3.722656,60.0,1.0,2.898438,6.0,0.0,12.523438,23.0,0.0,train,1187899
1,1,10258,9,1.000000,9.0,0.899902,10,5,1,5.898438,4,7,41.0,18.0,0.694824,1946,0.713867,45,4.277344,29.0,1.0,2.718750,6.0,0.0,12.257812,23.0,0.0,train,1187899
2,1,10326,1,0.166667,1.0,0.099976,10,5,1,5.898438,4,7,41.0,18.0,0.694824,5526,0.651855,131,4.191406,37.0,1.0,2.742188,6.0,0.0,12.898438,23.0,0.0,train,1187899


In [ ]:
train_data = final_data.loc[final_data['eval_set']=='train']
test_data = final_data.loc[final_data['eval_set']=='test']

In [ ]:
train_data = pd.merge(train_data, train[['product_id', 'order_id', 'reordered']], on=['product_id', 'order_id'], how='left')
train_data.head(3)

,user_id,product_id,times_bought,user_pro_reorder_ratio,recent_ten,recent_ten_ratio,max_number_of_orders,mean_number_of_orders,min_number_of_orders,average_user_product,max_order_day,max_order_by_hour,total_reorders,total_non_reorders,reorder_ratio,times_product_ordered,product_reorder_ratio,product_not_reorder_time,average_product_cart_position,late_product_cart_position,early_product_cart_position,average_product_order_day,late_product_order_day,early_product_order_day,average_product_hour,late_product_hour,early_product_hour,eval_set,order_id,reordered
0,1,196,10,1.000000,10.0,1.000000,10,5,1,5.898438,4,7,41.0,18.0,0.694824,35791,0.776367,64,3.722656,60.0,1.0,2.898438,6.0,0.0,12.523438,23.0,0.0,train,1187899,1.0
1,1,10258,9,1.000000,9.0,0.899902,10,5,1,5.898438,4,7,41.0,18.0,0.694824,1946,0.713867,45,4.277344,29.0,1.0,2.718750,6.0,0.0,12.257812,23.0,0.0,train,1187899,1.0
2,1,10326,1,0.166667,1.0,0.099976,10,5,1,5.898438,4,7,41.0,18.0,0.694824,5526,0.651855,131,4.191406,37.0,1.0,2.742188,6.0,0.0,12.898438,23.0,0.0,train,1187899,NaN


In [ ]:
train_data['reordered'].fillna(0, inplace=True)
print("Null values", train_data.isnull().any().sum().sum())

Null values 0


In [ ]:
train_data = train_data.set_index(['user_id', 'product_id'])
test_data = test_data.set_index(['user_id', 'product_id'])

train_data.drop(['eval_set', 'order_id'], axis=1, inplace=True)
test_data.drop(['eval_set', 'order_id'], axis=1, inplace=True)

In [ ]:
print("Train Data Shape", train_data.shape)
print("Test Data Shape", test_data.shape)

Train Data Shape (8474661, 26)
Test Data Shape (4833292, 25)


In [ ]:
pickle.dump((train_data),open('/content/drive/My Drive/Instacart Analysis/final_data_trainn.pkl','wb'))
pickle.dump((test_data),open('/content/drive/My Drive/Instacart Analysis/final_data_test.pkl','wb'))